In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("./sample_market_data.csv")
df["timestamp"] = pd.to_datetime(df["timestamp"])
df.head()


,orderId,price,qty,side,timestamp
0,1,27.65,22,ASK,2023-11-29 17:01:01.855124
1,2,27.25,88,ASK,2023-11-29 17:01:10.855124
2,3,27.06,28,ASK,2023-11-29 17:01:11.855124
3,4,27.21,47,ASK,2023-11-29 17:01:15.855124
4,1,27.65,12,ASK,2023-11-29 17:01:25.855124


In [4]:
# Defining a timestamp to be later used
#timestamp = "2023-11-29 17:01:25.855124"
timestamp = "2023-11-29 17:30:55.855124"
timestamp = pd.to_datetime(timestamp)

In [5]:
# Forming a list of orders at a specified timestamp
orders = df[df["timestamp"] <= timestamp]
orders.head()

,orderId,price,qty,side,timestamp
0,1,27.65,22,ASK,2023-11-29 17:01:01.855124
1,2,27.25,88,ASK,2023-11-29 17:01:10.855124
2,3,27.06,28,ASK,2023-11-29 17:01:11.855124
3,4,27.21,47,ASK,2023-11-29 17:01:15.855124
4,1,27.65,12,ASK,2023-11-29 17:01:25.855124


In [6]:
# Filtering to get the latest order for a corresponding orderid

latest_order_ids = orders.groupby("orderId")['timestamp'].idxmax()
orders = orders.loc[latest_order_ids]
orders = orders[orders["qty"] != 0]
orders.head()

,orderId,price,qty,side,timestamp
229,1,27.65,78,ASK,2023-11-29 17:21:18.855124
2,3,27.06,28,ASK,2023-11-29 17:01:11.855124
3,4,27.21,47,ASK,2023-11-29 17:01:15.855124
5,5,26.49,65,BID,2023-11-29 17:01:26.855124
6,6,27.47,10,ASK,2023-11-29 17:01:36.855124


In [7]:
# Forming the order book from the orders

order_book = orders.groupby(['side', 'price']).agg({'qty' : 'sum', 'orderId' : 'count'})
order_book.rename(columns={'orderId' : 'num_orders'}, inplace=True)
order_book.sort_values('price', ascending=False, inplace=True)
order_book = order_book.reset_index()

order_book.head()

,side,price,qty,num_orders
0,ASK,27.94,4,1
1,ASK,27.93,34,1
2,ASK,27.92,79,1
3,ASK,27.89,123,2
4,ASK,27.83,15,1


In [8]:
order_book.to_csv('order_book.csv')

In [9]:
# Additional calculations (not part of the task)
lowest_ask_idx = order_book.groupby("side")["price"].idxmin()[0]
highest_bid_idx = order_book.groupby("side")["price"].idxmax()[1]

spread = order_book.iloc[lowest_ask_idx]["price"] - order_book.iloc[highest_bid_idx]["price"]
print(spread)

0.029999999999997584


C:\Users\Zain\AppData\Local\Temp\ipykernel_23676\151278940.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lowest_ask_idx = order_book.groupby("side")["price"].idxmin()[0]
C:\Users\Zain\AppData\Local\Temp\ipykernel_23676\151278940.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  highest_bid_idx = order_book.groupby("side")["price"].idxmax()[1]
